In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

# -------------------------------------------------------------
# 1. Chargement des données
# -------------------------------------------------------------
hotCode='BEE'   

df = pd.read_csv('../data/'+hotCode+'/Indicateurs.csv', sep=';')
df.fillna(0, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])

df

,hotCode,Date,ObsDate,Chv,Clx,Sel,Libres,Construites,Ca,CaH,To,Pm,revpz,Gt,Te,Ant,Ds
0,BEE,2022-01-01,2021-11-02,0,0,0,48,48,0.00,0.00,0.0000,0.00,0.00,0,0,0.0,0.00
1,BEE,2022-01-01,2021-11-03,0,0,0,48,48,0.00,0.00,0.0000,0.00,0.00,0,0,0.0,0.00
2,BEE,2022-01-01,2021-11-04,0,0,0,48,48,0.00,0.00,0.0000,0.00,0.00,0,0,0.0,0.00
3,BEE,2022-01-01,2021-11-05,0,0,0,48,48,0.00,0.00,0.0000,0.00,0.00,0,0,0.0,0.00
4,BEE,2022-01-01,2021-11-06,0,0,0,48,48,0.00,0.00,0.0000,0.00,0.00,0,0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86859,BEE,2025-11-24,2025-11-20,42,11,53,6,48,7633.99,6977.98,0.8750,166.14,145.37,0,0,35.0,1.23
86860,BEE,2025-11-24,2025-11-21,43,14,57,5,48,7854.26,7144.65,0.8958,166.15,148.85,0,0,33.0,1.24
86861,BEE,2025-11-24,2025-11-22,46,14,60,2,48,8426.58,7667.05,0.9583,166.68,159.73,0,0,32.0,1.24
86862,BEE,2025-11-24,2025-11-23,51,15,66,-3,48,9420.60,8554.02,1.0625,167.73,178.21,0,0,28.0,1.25


In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


# Sélection des indicateurs utiles
X = df[['To', 'Pm', 'CaH', 'Ant', 'Chv', 'Libres']]

# Normalisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Clustering en 3 saisons
kmeans = KMeans(n_clusters=3, random_state=42)
df['SeasonCluster'] = kmeans.fit_predict(X_scaled)

# Interprétation des clusters (automatique)
cluster_summary = df.groupby('SeasonCluster')[['To','Pm','CaH']].mean()
print(cluster_summary)




                     To          Pm          CaH
SeasonCluster                                   
0              0.553867  144.375621  3691.199408
1              0.905583  212.159184  9101.003388
2              0.142423  120.144632   977.866538


Comment interpréter les clusters ?

Une fois le clustering fait :

Le cluster avec le TO le plus haut + PM élevé = Haute saison

Le cluster intermédiaire = Moyenne saison

Le cluster avec TO faible + PM faible = Basse saison

Exemple :

Cluster	TO	PM	Interprétation
2	0.87	210	Haute saison
0	0.52	140	Moyenne
1	0.27	90	Basse

In [6]:
df

,hotCode,Date,ObsDate,Chv,Clx,Sel,Libres,Construites,Ca,CaH,To,Pm,revpz,Gt,Te,Ant,Ds,SeasonCluster
0,BEE,2022-01-01,2021-11-02,0,0,0,48,48,0.00,0.00,0.0000,0.00,0.00,0,0,0.0,0.00,2
1,BEE,2022-01-01,2021-11-03,0,0,0,48,48,0.00,0.00,0.0000,0.00,0.00,0,0,0.0,0.00,2
2,BEE,2022-01-01,2021-11-04,0,0,0,48,48,0.00,0.00,0.0000,0.00,0.00,0,0,0.0,0.00,2
3,BEE,2022-01-01,2021-11-05,0,0,0,48,48,0.00,0.00,0.0000,0.00,0.00,0,0,0.0,0.00,2
4,BEE,2022-01-01,2021-11-06,0,0,0,48,48,0.00,0.00,0.0000,0.00,0.00,0,0,0.0,0.00,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86859,BEE,2025-11-24,2025-11-20,42,11,53,6,48,7633.99,6977.98,0.8750,166.14,145.37,0,0,35.0,1.23,1
86860,BEE,2025-11-24,2025-11-21,43,14,57,5,48,7854.26,7144.65,0.8958,166.15,148.85,0,0,33.0,1.24,1
86861,BEE,2025-11-24,2025-11-22,46,14,60,2,48,8426.58,7667.05,0.9583,166.68,159.73,0,0,32.0,1.24,1
86862,BEE,2025-11-24,2025-11-23,51,15,66,-3,48,9420.60,8554.02,1.0625,167.73,178.21,0,0,28.0,1.25,1
